# 데이터 읽어오기

In [68]:
import pandas as pd
movies = pd.read_csv("./dataset/tmdb_5000_movies.csv")         # tmdb_5000_movies.csv dataframe으로 읽어오기
movies = movies[["id","genres","vote_average","vote_count","popularity","original_title", "keywords","overview"]]
movies.drop(["genres","vote_average","vote_count","popularity", "keywords","overview"],axis=1,inplace=True)

print(movies.shape)
movies.head()

(4803, 2)


,id,original_title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter


In [69]:
# 컬럼명 변경
movies.columns = ['tmdbId', 'original_title']
movies.head()

,tmdbId,original_title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter


In [70]:
# links.csv dataframe으로 읽어오기
links = pd.read_csv("./dataset/links_small.csv")
links = links[["movieId","imdbId","tmdbId"]]
links.drop(['imdbId'],axis=1,inplace=True)

print(links.shape)
links.head()

(9125, 2)


,movieId,tmdbId
0,1,862.0
1,2,8844.0
2,3,15602.0
3,4,31357.0
4,5,11862.0


In [71]:
# ratings.csv dataframe으로 읽어오기
ratings = pd.read_csv("./dataset/ratings_small.csv")   
ratings = ratings[["userId","movieId","rating","timestamp"]]
ratings.drop(["timestamp"],axis=1,inplace=True)

print(ratings.shape)
ratings.head()

(100004, 3)


,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


## 결측값 확인


In [72]:
# null 값 개수 확인
movies.isnull().sum()

tmdbId            0
original_title    0
dtype: int64

In [73]:
# null 값 개수 확인
links.isnull().sum()

movieId     0
tmdbId     13
dtype: int64

In [74]:
# 결측값 있는 로우 제거
links = links.dropna(axis=0)

# 타입변환
links["tmdbId"] = links["tmdbId"].astype("int")

links.head()

,movieId,tmdbId
0,1,862
1,2,8844
2,3,15602
3,4,31357
4,5,11862


In [75]:
# null 값 개수 확인
ratings.isnull().sum()

userId     0
movieId    0
rating     0
dtype: int64

## 데이터 프레임끼리 inner join

In [76]:
join_movie = pd.merge(ratings, links, left_on='movieId', right_on='movieId', how='inner')
print(join_movie)

       userId  movieId  rating  tmdbId
0           1       31     2.5    9909
1           7       31     3.0    9909
2          31       31     4.0    9909
3          32       31     4.0    9909
4          36       31     3.0    9909
...       ...      ...     ...     ...
99928     664    64997     2.5   34812
99929     664    72380     3.5   22825
99930     665      129     3.0  110972
99931     665     4736     1.0   26602
99932     668     6425     1.0   36807

[99933 rows x 4 columns]


In [77]:
join_movie = pd.merge(join_movie, movies, left_on='tmdbId', right_on='tmdbId', how='inner')
print(join_movie)

       userId  movieId  rating  tmdbId  original_title
0           1     1061     3.0     819        Sleepers
1          19     1061     3.0     819        Sleepers
2          23     1061     3.5     819        Sleepers
3          30     1061     3.0     819        Sleepers
4          70     1061     5.0     819        Sleepers
...       ...      ...     ...     ...             ...
66942     663   134528     3.5  222936           Aloha
66943     663   137595     3.0  264999  Magic Mike XXL
66944     664    60832     3.0   12192       Pathology
66945     664    72380     3.5   22825         The Box
66946     665     4736     1.0   26602    Summer Catch

[66947 rows x 5 columns]


In [78]:
join_movie["original_title"].value_counts()

Forrest Gump                341
Pulp Fiction                324
The Shawshank Redemption    311
The Silence of the Lambs    304
Star Wars                   291
                           ... 
I, Frankenstein               1
Bandidas                      1
Def-Con 4                     1
Take Me Home Tonight          1
The Good German               1
Name: original_title, Length: 3393, dtype: int64

In [79]:
# # movies에 있던 모든 영화가 join이 잘 됐는지 확인 -> 여긴 발표할 때는 빼도 될 듯..
# after = join_movie["tmdbId"].unique()
# print(len(after))
# print(movies.shape)

## latent 알고리즘

### 피벗테이블 생성

In [80]:
user_movie_rating = join_movie.pivot_table('rating', index = 'userId', columns='tmdbId').fillna(0)
print(user_movie_rating.shape)
user_movie_rating.head()

(671, 3394)


tmdbId,5,11,12,13,14,16,18,19,20,22,24,25,28,33,35,38,55,58,59,62,65,66,68,69,70,71,73,74,75,76,77,78,79,80,83,85,87,89,90,95,...,291270,291805,293660,293863,294086,294254,295964,296096,296098,296099,297596,297761,298312,301748,302699,303858,307081,308531,308639,310131,312221,313922,314365,316727,318846,321258,321697,321741,324668,325133,325173,328111,328425,329833,332411,332567,333371,334074,342521,347969
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,3.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,5.0,5.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 특이값 분해 (Singular Value Decomposion, SVD)
행렬을 여러 행렬의 곱으로 분해해준다. </br>
[참고 사이트](https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/04/06/pcasvdlsa/)

scipy의 Truncated SVD 이용 </br>

반환값: U 행렬, Sigma 행렬, V 전치 행렬(Vt)



In [81]:
from scipy.sparse.linalg import svds
import numpy as np

# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(user_movie_rating, k = 670)


print(U.shape)
print(sigma.shape)
print(Vt.shape)

(671, 670)
(670,)
(670, 3394)



현재 이 Sigma 행렬은 0이 아닌 값만 1차원 행렬로 표현된 상태->0이 포함된 대칭행렬로 변환할 때는 numpy의 diag를 이용

In [82]:
sigma = np.diag(sigma)

tmp = pd.DataFrame(data=sigma, index=None, columns=None, dtype=None, copy=False)
print(tmp.shape)
tmp.head()

(670, 670)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669
0,1.438755,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,1.513445,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,1.639906,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,1.744034,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.800951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 원래 행렬로 복구

U, Sigma, Vt의 내적을 수행

In [83]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 

svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 


In [84]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = user_movie_rating.columns)
print(df_svd_preds.shape)
df_svd_preds.head()

(671, 3394)


tmdbId,5,11,12,13,14,16,18,19,20,22,24,25,28,33,35,38,55,58,59,62,65,66,68,69,70,71,73,74,75,76,77,78,79,80,83,85,87,89,90,95,...,291270,291805,293660,293863,294086,294254,295964,296096,296098,296099,297596,297761,298312,301748,302699,303858,307081,308531,308639,310131,312221,313922,314365,316727,318846,321258,321697,321741,324668,325133,325173,328111,328425,329833,332411,332567,333371,334074,342521,347969
0,0.000056,0.000076,-0.000007,0.000096,-9.427093e-06,-0.000026,0.000072,-0.000078,2.037244e-06,-0.000095,0.000031,0.000038,-0.000023,0.000102,0.000037,0.000066,-0.000019,-0.000015,-0.000013,0.000012,-0.000050,3.519704e-06,0.000078,-0.000027,-0.000011,-0.000038,0.000027,-0.000034,2.161966e-06,0.000139,0.000051,-0.000029,-0.000011,-0.000041,0.000025,-0.000093,0.000014,4.877570e-06,0.000035,-0.000027,...,2.627405e-06,-8.048095e-06,0.000024,3.941108e-06,-0.000012,-0.000020,-5.402025e-07,-1.093997e-06,0.000011,-0.000012,-4.051519e-07,-8.103037e-07,2.276696e-06,-4.051519e-07,3.284257e-06,1.970554e-06,-7.479684e-06,2.627405e-06,-2.847820e-06,-3.903165e-06,0.000015,3.118016e-07,0.000019,2.627405e-06,0.000016,2.276696e-06,3.468061e-06,1.313703e-06,6.813699e-06,2.734861e-08,-8.103037e-07,2.197052e-07,4.707207e-06,0.000022,-4.051519e-07,4.597959e-06,1.807710e-06,-5.002036e-06,1.313703e-06,1.069648e-06
1,-0.001415,-0.001928,0.000175,2.997566,2.389307e-04,0.000671,-0.001828,0.001987,-5.163416e-05,0.002414,-0.000774,-0.000975,0.000587,-0.002598,-0.000931,-0.001675,0.000474,0.000372,0.000333,-0.000310,0.001263,-8.920726e-05,-0.001985,0.000681,0.000285,0.000974,-0.000691,0.000854,-5.479525e-05,-0.003518,-0.001298,0.000724,0.000279,0.001041,-0.000629,0.002347,-0.000348,-1.236225e-04,-0.000897,0.000696,...,-6.659186e-05,2.039798e-04,-0.000618,-9.988780e-05,0.000312,0.000501,1.369149e-05,2.772748e-05,-0.000287,0.000307,1.026862e-05,2.053723e-05,-5.770309e-05,1.026862e-05,-8.323983e-05,-4.994390e-05,1.895734e-04,-6.659186e-05,7.217829e-05,9.892614e-05,-0.000380,-7.902645e-06,-0.000490,-6.659186e-05,-0.000408,-5.770309e-05,-8.789836e-05,-3.329593e-05,-1.726939e-04,-6.931533e-07,2.053723e-05,-5.568451e-06,-1.193046e-04,-0.000551,1.026862e-05,-1.165358e-04,-4.581661e-05,1.267771e-04,-3.329593e-05,-2.711033e-05
2,0.000103,0.000141,2.999987,5.000177,3.999983e+00,-0.000049,0.000133,-0.000145,3.763230e-06,-0.000176,0.000056,0.000071,-0.000043,0.000189,0.000068,3.000122,-0.000035,-0.000027,-0.000024,0.000023,-0.000092,6.501654e-06,0.000145,-0.000050,-0.000021,-0.000071,0.000050,-0.000062,3.993619e-06,0.000256,0.000095,-0.000053,-0.000020,-0.000076,0.000046,-0.000171,0.000025,9.009928e-06,0.000065,-0.000051,...,4.853386e-06,-1.486657e-05,0.000045,7.280079e-06,-0.000023,-0.000037,-9.978709e-07,-2.020850e-06,0.000021,-0.000022,-7.484031e-07,-1.496806e-06,4.205549e-06,-7.484031e-07,6.066733e-06,3.640040e-06,-1.381660e-05,4.853386e-06,-5.260539e-06,-7.209991e-06,0.000028,5.759651e-07,0.000036,4.853386e-06,0.000030,4.205549e-06,6.406258e-06,2.426693e-06,1.258638e-05,5.051879e-08,-1.496806e-06,4.058430e-07,8.695229e-06,0.000040,-7.484031e-07,8.493426e-06,3.339232e-06,-9.239842e-06,2.426693e-06,1.975870e-06
3,0.000020,5.000028,-0.000003,5.000035,-3.446955e-06,-0.000010,0.000026,-0.000029,7.449050e-07,-0.000035,0.000011,0.000014,4.999992,0.000037,0.000013,0.000024,-0.000007,-0.000005,-0.000005,0.000004,-0.000018,1.286957e-06,0.000029,-0.000010,-0.000004,-0.000014,0.000010,-0.000012,7.905088e-07,0.000051,0.000019,4.999990,-0.000004,-0.000015,0.000009,4.999966,5.000005,5.000002e+00,0.000013,3.999990,...,9.606936e-07,-2.942733e-06,0.000009,1.441040e-06,-0.000005,-0.000007,-1.975215e-07,-4.000130e-07,0.000004,-0.000004,-1.481411e-07,-2.962823e-07,8.324589e-07,-1.481411e-07,1.200867e-06,7.205202e-07,-2.734898e-06,9.606936e-07,-1.041287e-06,-1.427167e-06,0.000005,1.140082e-07,0.000007,9.606936e-07,0.000006,8.324589e-07,1.268074e-06,4.803468e-07,2.491385e-06,9.999840e-09,-2.962823e-07,8.033377e-08,1.721159e-06,0.000008,-1.481411e-07,1.681214e-06,6.609776e-07,-1.828962